# Australian Energy Production Data Pipeline

In [1]:
import pandas as pd
import requests
import json
import os
from sqlalchemy import create_engine

In [2]:
api_url = "https://visualisations.aemo.com.au/aemo/apps/api/report/5MIN"
payload = json.dumps({"timeScale":["5MIN"]})
headers = {'Content-Type':'application/json'}

In [3]:
db_url = os.getenv("DB_URL")
engine = create_engine(db_url)

In [3]:
response = requests.request("POST",api_url,headers=headers,data=payload)
data = response.json()

In [4]:
eGen = pd.DataFrame(data["5MIN"])

In [7]:
state_dfs = {state: pd.DataFrame(columns=eGen.columns) for state in eGen['REGION'].unique()}
for state in state_dfs.keys():
    state_dfs[state] = eGen[eGen['REGION'] == state]
    state_dfs[state].to_sql(f'electricity_data_{state}', con=engine, if_exists='append', index=False)

In [5]:
eGen.groupby('REGION').tail(1)

,SETTLEMENTDATE,REGIONID,REGION,RRP,TOTALDEMAND,PERIODTYPE,NETINTERCHANGE,SCHEDULEDGENERATION,SEMISCHEDULEDGENERATION,APCFLAG
575,2024-07-17T17:25:00,NSW1,NSW1,288.55501,10749.29,ACTUAL,-901.28,9210.74240,620.52760,0.0
1151,2024-07-17T17:25:00,QLD1,QLD1,302.26313,8400.93,ACTUAL,198.84,8282.71913,321.95087,0.0
1727,2024-07-17T17:25:00,SA1,SA1,229.98000,2142.96,ACTUAL,340.14,1078.69055,1401.06945,0.0
2303,2024-07-17T17:25:00,TAS1,TAS1,270.22394,1507.24,ACTUAL,-320.40,1088.80001,98.03999,0.0
2879,2024-07-17T17:25:00,VIC1,VIC1,256.31000,7237.16,ACTUAL,725.84,6409.39820,1525.65180,0.0
